In [1]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

In [2]:
env = gym.make("Pendulum-v1", render_mode="rgb_array")

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = TD3("MlpPolicy", env, action_noise=action_noise, verbose=1)
# "MlpPolicy",
# "CnnPolicy",
# "MultiInputPolicy",

model.learn(total_timesteps=10000, log_interval=10)
# model.save("td3_pendulum")
vec_env = model.get_env()

# del model # remove to demonstrate saving and loading

# model = TD3.load("td3_pendulum")

obs = vec_env.reset()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 200       |
|    ep_rew_mean     | -1.46e+03 |
| time/              |           |
|    episodes        | 10        |
|    fps             | 116       |
|    time_elapsed    | 17        |
|    total_timesteps | 2000      |
| train/             |           |
|    actor_loss      | 33.3      |
|    critic_loss     | 0.143     |
|    learning_rate   | 0.001     |
|    n_updates       | 1899      |
----------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 200      |
|    ep_rew_mean     | -1.2e+03 |
| time/              |          |
|    episodes        | 20       |
|    fps             | 110      |
|    time_elapsed    | 36       |
|    total_timesteps | 4000     |
| train/             |          |
|    actor_loss      | 51.6     |
|    c

In [ ]:

model = TD3("MultiInputPolicy", env, action_noise=action_noise, verbose=1)
# "MlpPolicy",
# "CnnPolicy",
# "MultiInputPolicy",

model.learn(total_timesteps=10000, log_interval=10)
# model.save("td3_pendulum")
vec_env = model.get_env()

# del model # remove to demonstrate saving and loading

# model = TD3.load("td3_pendulum")

obs = vec_env.reset()

In [ ]:
import gymnasium as gym
import numpy as np

from stable_baselines3 import TD3
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise


while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

In [3]:
from omegaconf import OmegaConf
from octoplus.src.config_dataclasses.experiments import ExperimentConfig

# Load configuration from a file
config = OmegaConf.load('/home/piscenco/thesis_octo/thesis-code/configs/eval_config.yaml')
print(config.model_type)
print(config.model_config)
del config['defaults']
del config['hydra']
# Print the configuration
# config = OmegaConf.to_yaml(config)

config = ExperimentConfig(**config)

2024-12-09 23:39:43,884 - root - INFO - Casting model_config to PPORGBConfig type
2024-12-09 23:39:43,889 - root - INFO - Casting env_config to ManiSkillConfig type
2024-12-09 23:39:43,889 - root - INFO - Scheduling scheduling_strategy was set to internal_only.


ppo_rgb
{'exp_name': 'evaluation', 'model_device': 'cuda', 'seed': 13, 'num_steps': '${env_config.num_steps}', 'num_envs': '${env_config.num_envs}', 'include_state': False, 'total_timesteps': 5000, 'learning_rate': 0.0003, 'num_minibatches': 10, 'actor_type': 'BasicActor', 'actor_activation_layer': 'ReLU', 'critic_type': 'BasicCritic', 'critic_activation_layer': 'ReLU'}


In [ ]:
from octoplus.src.env.mani_skill_env import get_envs

envs = get_envs(
        env_config=config.env_config,
    )

/home/piscenco/thesis_octo/octoplus_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-12-09 23:40:10,161 - octoplus.src.env.mani_skill_env - INFO - Using render camera as input.
/home/piscenco/thesis_octo/octoplus_env/lib/python3.10/site-packages/torch/random.py:183: UserWarning: CUDA reports that you have 2 available devices, and you have used fork_rng without explicitly specifying which devices are being used. For safety, we initialize *every* CUDA device by default, which can be quite slow if you have a lot of CUDAs. If you know that you are only making use of a few CUDA devices, set the environment variable CUDA_VISIBLE_DEVICES or the 'devices' keyword argument of fork_rng with the set of devices you are actually using. For example, if you are using CPU only, set device.upper()_VISIBLE_DEVICE

env action_space: Box(-1.0, 1.0, (1, 7), float32)
env observation_space: Dict('rgb': Box(0, 255, (1, 256, 256, 3), uint8))


/home/piscenco/thesis_octo/octoplus_env/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_observation_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_observation_space` for environment variables or `env.get_wrapper_attr('single_observation_space')` that will search the reminding wrappers.
  logger.warn(
/home/piscenco/thesis_octo/octoplus_env/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.single_action_space to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.single_action_space` for environment variables or `env.get_wrapper_attr('single_action_space')` that will search the reminding wrappers.
  logger.warn(


In [17]:
config.env_config

ManiSkillConfig(obs_mode='rgb', control_mode='pd_ee_delta_pose', render_mode='rgb_array', sim_backend='gpu', reward_mode='dense', env_id='PickCube-v1', num_envs=1, num_steps=50, include_state=False, partial_reset=True, image_height=256, image_width=256, use_render_camera_as_input=True)

In [11]:
from mani_skill.envs.sapien_env import BaseEnv

class ToCPUWrapper(gym.ObservationWrapper):
    """
    Convert the observation to CPU
    """

    def __init__(self, env) -> None:
        self.base_env: BaseEnv = env.unwrapped
        super().__init__(env)
        new_obs = self.observation(self.base_env._init_raw_obs)
        self.base_env.update_obs_space(new_obs)

    def observation(self, observation: dict):
        observation["rgb"] = observation["rgb"].cpu()
        return observation

In [13]:
envs = ToCPUWrapper(envs)

In [16]:
model2 = TD3("MultiInputPolicy", envs, action_noise=action_noise, verbose=1)
# "MlpPolicy",
# "CnnPolicy",
# "MultiInputPolicy",

model2.learn(total_timesteps=100, log_interval=10)
# model.save("td3_pendulum")
# vec_env = model2.get_env()

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/home/piscenco/thesis_octo/octoplus_env/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:605: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 393.25GB > 242.02GB
  warnings.warn(


TypeError: argument of type 'NoneType' is not iterable